In [58]:
import pandas as pd
df = pd.read_csv('/home/gordon/work/project_models/build/test/sklearnflask/data/titanic.csv')
include = ['Age', 'Sex', 'Embarked', 'Survived']
df_ = df[include]  # only using 4 variables

In [59]:
categoricals = []
for col, col_type in df_.dtypes.iteritems():
     if col_type == 'O':
          categoricals.append(col)
     else:
          df_[col].fillna(0, inplace=True)

In [60]:
df_ohe = pd.get_dummies(df, columns=categoricals, dummy_na=True)
df_ohe = df_ohe.drop(['Name', 'Ticket', 'Cabin'], axis = 1)
df_ohe = df_ohe.fillna(0)
df_ohe.head(2)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Survived,Sex_female,Sex_male,Sex_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,1,3,22.0,1,0,7.2500,0,0,1,0,0,0,1,0
1,2,1,38.0,1,0,71.2833,1,1,0,0,1,0,0,0


In [61]:
# using a random forest classifier (can be any classifier)
from sklearn.ensemble import RandomForestClassifier as rf

dependent_variable = 'Survived'

y = df_ohe.pop('Survived')
X = df_ohe

clf = rf()
clf.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [62]:
from sklearn.externals import joblib
joblib.dump(clf, 'model.pkl')

clf = joblib.load('model.pkl')

['model.pkl']

In [69]:
import werkzeug
werkzeug.cached_property = werkzeug.utils.cached_property
from werkzeug.datastructures import FileStorage

from flask import Flask
from flask_restplus import Api, Resource, reqparse


app = Flask(__name__)
# api = Api(app)
#           version='10.5',
#           title='Flask Fraud App',
#           description='Predict Fraud Occurences',
#           contact='Ann Strange, Casey Olson, Joseph Wilson, Gordon Pisciotta',
#           )

@app.route('/', methods=['GET'])
def home():
    return '''     Hello And Welcome To Our Home Page!
                    We are so happy to welcome you.
                    In order to proceed please select the secrets
                    you will be prompted to enter your credentials!
                    <a href="/predict">predict</a>   '''

@app.route('/predict', methods=['POST'])
def predict():
     json_ = request.json
     query_df = pd.DataFrame(json_)
     query = pd.get_dummies(query_df)
     prediction = clf.predict(query)
     return jsonify({'prediction': list(prediction)})


if __name__ == '__main__':
     clf = joblib.load('/home/gordon/work/project_models/build/test/sklearnflask/model/model.pkl')
     app.run(port=8079)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:8079/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Dec/2020 05:34:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2020 05:34:05] "GET /swagger.json HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2020 05:34:05] "GET /swaggerui/favicon-32x32.png HTTP/1.1" 200 -


NameError: name 'x' is not defined